### CPU 환경

In [ ]:
!lscpu

### GPU 환경

In [ ]:
!nvidia-smi

### RAM 용량 확인

In [ ]:
!free -h

### HDD 용량 확인

In [ ]:
# 디스크 용량 확인
!df -h

### OS 환경

In [ ]:
!cat /etc/os-release

## 패키지 불러오기

In [ ]:
pip install torchinfo

In [18]:
import os
import os.path as osp
import sys
from glob import glob

import datetime
import time
import pytz

import pandas as pd
import numpy as np
from tqdm import tqdm

import sklearn

import torch
import torchvision
from torchinfo import summary

import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
import torchvision.models as models

# import explain

In [ ]:
print('python version:',sys.version)
print('pandas version:',pd.__version__)
print('numpy version:', np.__version__)

print('sklearn version:', sklearn.__version__)
print('torch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)

## 모델 파라미터 (Parameter)

In [22]:
# 모델에 입력할 Input 이미지의 크기를 지정함.
IMG_WIDTH = 512
IMG_HEIGHT = 512

# 배치 사이즈 임의 지정
batch_size = 4

# 데이터를 읽어올 경로를 지정함.
data_dir= '/content/drive/MyDrive/project_files/New_sample/Dataset/preprocessed/classification/'
# 학습된 모델이 저장된 경로를 지정함.
ckpt_dir = '/content/drive/MyDrive/project/QC_cow1/classification/model_weight/'

# 모델 이름
Model_name = 'resnext50_32x4d_cow_classification_v1_best_Loss'
# 모델을 연산할 장비 설정 (Docker환경에서 작동할 수 있도록 CPU에서 계산함)
device = torch.device('cpu')

## Train, Validation, Test 데이터 셋 확인

In [ ]:
label=['1++','1+','1','2','3']

for set_ in ['train','val','test_1']:
    set_size=len(glob(osp.join(data_dir, set_, '*', '*'), recursive=True))
    if set_=='val':
        print(f"{set_+'idation'} set size: {set_size}\n")
    else:
        print(f"{set_} set size: {set_size}\n")
    
    for i in range(5):
        size=len(glob(osp.join(data_dir, set_, str(i), '*'), recursive=True))
        print(f"The number of Class {label[i]}: {size} / {set_size} = {size/set_size :.2f}")
    print('-'*45)
    print()

## 데이터 셋 (Dataset) 불러오기

In [24]:
# 이미지 transformation
test_compose=transforms.Compose([
    transforms.Resize((IMG_HEIGHT,IMG_WIDTH)),
    transforms.ToTensor()
])

# Test set을 읽어옴
test_dataset=datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_compose)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)

### 모델 불러오기 (Load)

In [ ]:
# Model: ResNeXt50_32x4d
model=models.resnext50_32x4d(pretrained=False)
# 최종 Label 의 개수 = 5 (1++, 1+, 1, 2, 3)
model.fc=nn.Linear(model.fc.in_features, 5)

# 학습된 모델 가중치 불러오기
checkpoint=torch.load(ckpt_dir + Model_name + '.pth', map_location=device)
model.load_state_dict(checkpoint['state_dict'])

### 모델 학습 전, ImageNet 데이터로 Pretrain 된 기본 모델 불러오기 (Load)

In [ ]:
# model : ResNeXt50_32x4d
base_model=models.resnext50_32x4d(pretrained=True)
base_model.fc=nn.Linear(base_model.fc.in_features, 5)

## 모델 확인 (Summary)

In [ ]:
_ = summary(model, (4, 3, 512, 512))

_

---

## 테스트 (Test) 수행

### 1. ImageNet 데이터로 Pretrain 된 기본 모델 성능 평가

### 테스트 이미지 평가 (Predict)

In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

In [ ]:
since=time.time()

In [ ]:
tot_labels=[]  # 전체 정답 Label 을 저장할 리스트
tot_pred_labels=[]  # 전체 예측 Label 을 저장할 리스트

# 모델을 device에 올림
base_model=base_model.to(device)
# 네트워크를 evaluation 용으로 선언
base_model=base_model.eval()

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):

        images=images.to(device)
        labels=labels.to(device)

        # Model Prediction (forward)
        output=base_model(images)
        _, output_index = torch.max(output, 1)    

        tot_labels.extend(list(labels.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).numpy()))

In [ ]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 2m 41s


In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

## Accuracy

In [ ]:
result = explain.ShowResult(tot_labels, tot_pred_labels)

In [ ]:
result.show_result()

---

### 2. 기본 모델을 Fine Tuning (학습) 한 모델 성능 평가 (Transfer Learning)

### 테스트 이미지 평가 (Predict)

In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

In [ ]:
since=time.time()

In [ ]:
tot_labels=[]  # 전체 정답 Label 을 저장할 리스트
tot_pred_labels=[]  # 전체 예측 Label 을 저장할 리스트

# 모델을 device에 올림
model=model.to(device)
# 네트워크를 evaluation 용으로 선언
model=model.eval()

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):

        images=images.to(device)
        labels=labels.to(device)

        # Model Prediction (forward)
        output=model(images)
        _, output_index = torch.max(output, 1)    

        tot_labels.extend(list(labels.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).numpy()))

In [ ]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

## Accuracy

In [ ]:
result=explain.ShowResult(tot_labels, tot_pred_labels)

In [ ]:
result.show_result()

Predicted  1++  1+  1  2   3  All
True                             
1++         19   0  1  0   0   20
1+          17   0  3  0   0   20
1           14   2  2  0   2   20
2           10   0  2  1   7   20
3            1   0  0  4  15   20
All         61   2  8  5  24  100


#-- Confusion Matrix for class 1++

                Pred    
             Non 1++ 1++
True Non 1++      38  42
     1++           1  19

Accuracy for class 1++ : 57.0
-----------------------------------

#-- Confusion Matrix for class 1+

              Pred   
            Non 1+ 1+
True Non 1+     78  2
     1+         20  0

Accuracy for class 1+ : 78.0
-----------------------------------

#-- Confusion Matrix for class 1

            Pred   
           Non 1  1
True Non 1    74  6
     1        18  2

Accuracy for class 1 : 76.0
-----------------------------------

#-- Confusion Matrix for class 2

            Pred   
           Non 2  2
True Non 2    76  4
     2        19  1

Accuracy for class 2 : 77.0
-----------------------------------

#-- Confusion Matrix for class 3

            Pred    
           Non 3   3
True Non 3    71   9
     3         5  15

Accuracy for class 3 : 86.0
-----------------------------------

#-- Final Average Accuracy
( 57.0 + 78.0 + 76.0 + 77.0 + 86.0 ) / 5 = 74.800